# MetricaX Bayesian Statistics Demo 🎯

**Interactive walkthrough of Bayesian inference with real-world applications**

This notebook demonstrates the power of MetricaX's Bayesian statistics module through practical examples and visualizations.

In [ ]:
# Install MetricaX if not already installed
# !pip install metricax

import metricax.bayesian as mb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for beautiful plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ MetricaX Bayesian module loaded successfully!")
print(f"📊 Available functions: {len(mb.__all__)}")

## 🧪 **A/B Testing with Bayesian Analysis**

Let's analyze a real A/B test scenario where we're comparing conversion rates between two website variants.

In [ ]:
# A/B Test Data
control_visitors = 1000
control_conversions = 85

treatment_visitors = 1000  
treatment_conversions = 110

print(f"🅰️ Control: {control_conversions}/{control_visitors} = {control_conversions/control_visitors:.1%}")
print(f"🅱️ Treatment: {treatment_conversions}/{treatment_visitors} = {treatment_conversions/treatment_visitors:.1%}")
print(f"📈 Observed lift: {(treatment_conversions/treatment_visitors - control_conversions/control_visitors)/(control_conversions/control_visitors):.1%}")

In [ ]:
# Bayesian Analysis with Beta-Binomial Conjugate Priors
# Start with uninformative prior: Beta(1, 1)
prior_alpha, prior_beta = 1, 1

# Update with observed data
control_alpha, control_beta = mb.update_beta_binomial(
    prior_alpha, prior_beta, 
    control_conversions, control_visitors - control_conversions
)

treatment_alpha, treatment_beta = mb.update_beta_binomial(
    prior_alpha, prior_beta,
    treatment_conversions, treatment_visitors - treatment_conversions
)

print(f"📊 Posterior Distributions:")
print(f"   Control: Beta({control_alpha}, {control_beta})")
print(f"   Treatment: Beta({treatment_alpha}, {treatment_beta})")

# Calculate posterior statistics
control_mean = mb.beta_mean(control_alpha, control_beta)
treatment_mean = mb.beta_mean(treatment_alpha, treatment_beta)
control_var = mb.beta_var(control_alpha, control_beta)
treatment_var = mb.beta_var(treatment_alpha, treatment_beta)

print(f"\n📈 Posterior Estimates:")
print(f"   Control: {control_mean:.1%} ± {1.96*np.sqrt(control_var):.1%}")
print(f"   Treatment: {treatment_mean:.1%} ± {1.96*np.sqrt(treatment_var):.1%}")

In [ ]:
# Visualize the posterior distributions
x = np.linspace(0, 0.2, 1000)

# Calculate PDFs
control_pdf = [mb.beta_pdf(xi, control_alpha, control_beta) for xi in x]
treatment_pdf = [mb.beta_pdf(xi, treatment_alpha, treatment_beta) for xi in x]

# Create the plot
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(x, control_pdf, label='Control', linewidth=3, alpha=0.8)
plt.plot(x, treatment_pdf, label='Treatment', linewidth=3, alpha=0.8)
plt.axvline(control_mean, color='blue', linestyle='--', alpha=0.7, label=f'Control Mean: {control_mean:.1%}')
plt.axvline(treatment_mean, color='orange', linestyle='--', alpha=0.7, label=f'Treatment Mean: {treatment_mean:.1%}')
plt.xlabel('Conversion Rate')
plt.ylabel('Probability Density')
plt.title('Posterior Distributions of Conversion Rates')
plt.legend()
plt.grid(True, alpha=0.3)

# Monte Carlo simulation for probability that treatment > control
n_samples = 100000
control_samples = np.random.beta(control_alpha, control_beta, n_samples)
treatment_samples = np.random.beta(treatment_alpha, treatment_beta, n_samples)

prob_treatment_better = np.mean(treatment_samples > control_samples)
lift_samples = (treatment_samples - control_samples) / control_samples

plt.subplot(2, 2, 2)
plt.hist(lift_samples, bins=50, alpha=0.7, density=True, color='green')
plt.axvline(np.mean(lift_samples), color='red', linestyle='--', linewidth=2, label=f'Mean Lift: {np.mean(lift_samples):.1%}')
plt.xlabel('Relative Lift')
plt.ylabel('Density')
plt.title('Distribution of Relative Lift')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
credible_intervals = [50, 80, 95, 99]
lift_percentiles = []
for ci in credible_intervals:
    lower = (100 - ci) / 2
    upper = 100 - lower
    lift_lower = np.percentile(lift_samples, lower)
    lift_upper = np.percentile(lift_samples, upper)
    lift_percentiles.append((lift_lower, lift_upper))
    plt.barh(ci, lift_upper - lift_lower, left=lift_lower, alpha=0.6, label=f'{ci}% CI')

plt.xlabel('Relative Lift')
plt.ylabel('Credible Interval (%)')
plt.title('Credible Intervals for Lift')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
decision_metrics = {
    'P(Treatment > Control)': prob_treatment_better,
    'Expected Lift': np.mean(lift_samples),
    '95% CI Lower': np.percentile(lift_samples, 2.5),
    '95% CI Upper': np.percentile(lift_samples, 97.5)
}

metrics_text = "\n".join([f"{k}: {v:.1%}" if 'P(' in k or 'Lift' in k or 'CI' in k else f"{k}: {v:.3f}" 
                         for k, v in decision_metrics.items()])
plt.text(0.1, 0.5, metrics_text, fontsize=12, verticalalignment='center', 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.axis('off')
plt.title('Decision Metrics')

plt.tight_layout()
plt.show()

print(f"\n🎯 **Decision Analysis:**")
print(f"   Probability Treatment > Control: {prob_treatment_better:.1%}")
print(f"   Expected Relative Lift: {np.mean(lift_samples):.1%}")
print(f"   95% Credible Interval: [{np.percentile(lift_samples, 2.5):.1%}, {np.percentile(lift_samples, 97.5):.1%}]")

if prob_treatment_better > 0.95:
    print("\n✅ **Recommendation: Implement Treatment** (High confidence)")
elif prob_treatment_better > 0.8:
    print("\n⚠️ **Recommendation: Likely implement Treatment** (Moderate confidence)")
else:
    print("\n🤔 **Recommendation: Continue testing** (Insufficient evidence)")

## 🔄 **Online Learning: Quality Control**

Demonstrate how Bayesian methods excel at online learning scenarios where beliefs are updated as new data arrives.

In [ ]:
# Simulate a quality control scenario
# Initial belief: defect rate around 2%
alpha, beta = 2, 98
batches = [(100, 3), (150, 2), (200, 5), (120, 1), (180, 4), (90, 6), (110, 2)]

# Track evolution of beliefs
evolution = []
cumulative_items = 0
cumulative_defects = 0

for i, (items, defects) in enumerate(batches):
    cumulative_items += items
    cumulative_defects += defects
    
    # Update beliefs
    alpha, beta = mb.update_beta_binomial(alpha, beta, defects, items - defects)
    
    current_rate = mb.beta_mean(alpha, beta)
    current_var = mb.beta_var(alpha, beta)
    
    evolution.append({
        'batch': i + 1,
        'items': items,
        'defects': defects,
        'cumulative_items': cumulative_items,
        'cumulative_defects': cumulative_defects,
        'alpha': alpha,
        'beta': beta,
        'estimated_rate': current_rate,
        'uncertainty': np.sqrt(current_var),
        'observed_rate': cumulative_defects / cumulative_items
    })

# Visualize the evolution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Evolution of rate estimate
batches_num = [e['batch'] for e in evolution]
estimated_rates = [e['estimated_rate'] for e in evolution]
uncertainties = [e['uncertainty'] for e in evolution]
observed_rates = [e['observed_rate'] for e in evolution]

axes[0, 0].plot(batches_num, estimated_rates, 'o-', label='Bayesian Estimate', linewidth=2, markersize=8)
axes[0, 0].fill_between(batches_num, 
                       [r - 1.96*u for r, u in zip(estimated_rates, uncertainties)],
                       [r + 1.96*u for r, u in zip(estimated_rates, uncertainties)],
                       alpha=0.3, label='95% Credible Interval')
axes[0, 0].plot(batches_num, observed_rates, 's--', label='Observed Rate', alpha=0.7)
axes[0, 0].axhline(0.03, color='red', linestyle=':', label='Alert Threshold (3%)')
axes[0, 0].set_xlabel('Batch Number')
axes[0, 0].set_ylabel('Defect Rate')
axes[0, 0].set_title('Evolution of Defect Rate Estimates')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Uncertainty reduction
axes[0, 1].plot(batches_num, uncertainties, 'ro-', linewidth=2, markersize=8)
axes[0, 1].set_xlabel('Batch Number')
axes[0, 1].set_ylabel('Uncertainty (Standard Deviation)')
axes[0, 1].set_title('Uncertainty Reduction Over Time')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Posterior distributions evolution
x_range = np.linspace(0, 0.08, 1000)
colors = plt.cm.viridis(np.linspace(0, 1, len(evolution)))

for i, (evo, color) in enumerate(zip(evolution, colors)):
    if i % 2 == 0:  # Show every other batch to avoid clutter
        pdf_vals = [mb.beta_pdf(x, evo['alpha'], evo['beta']) for x in x_range]
        axes[1, 0].plot(x_range, pdf_vals, color=color, alpha=0.8, 
                       label=f"Batch {evo['batch']}")

axes[1, 0].set_xlabel('Defect Rate')
axes[1, 0].set_ylabel('Probability Density')
axes[1, 0].set_title('Evolution of Posterior Distributions')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Summary statistics
final_stats = evolution[-1]
stats_text = f"""Final Results (After {final_stats['cumulative_items']} items):

Estimated Defect Rate: {final_stats['estimated_rate']:.2%}
95% Credible Interval: [{final_stats['estimated_rate'] - 1.96*final_stats['uncertainty']:.2%}, 
{final_stats['estimated_rate'] + 1.96*final_stats['uncertainty']:.2%}]

Observed Defect Rate: {final_stats['observed_rate']:.2%}
Total Defects: {final_stats['cumulative_defects']}

Posterior: Beta({final_stats['alpha']:.0f}, {final_stats['beta']:.0f})"""

axes[1, 1].text(0.05, 0.5, stats_text, fontsize=11, verticalalignment='center',
                bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgreen", alpha=0.8))
axes[1, 1].set_xlim(0, 1)
axes[1, 1].set_ylim(0, 1)
axes[1, 1].axis('off')
axes[1, 1].set_title('Final Summary')

plt.tight_layout()
plt.show()

print("\n🏭 **Quality Control Analysis Complete!**")
print(f"   Final defect rate estimate: {final_stats['estimated_rate']:.2%}")
print(f"   Uncertainty: ±{1.96*final_stats['uncertainty']:.2%} (95% CI)")
if final_stats['estimated_rate'] > 0.03:
    print("   🚨 **ALERT**: Defect rate exceeds 3% threshold!")
else:
    print("   ✅ **OK**: Defect rate within acceptable limits")

## 🎓 **Key Takeaways**

This demo showcased the power of MetricaX's Bayesian statistics module:

1. **🎯 A/B Testing**: Rigorous statistical analysis with uncertainty quantification
2. **🔄 Online Learning**: Beliefs update naturally as new data arrives
3. **📊 Visualization**: Clear communication of statistical results
4. **💼 Business Impact**: Actionable insights for decision-making

### **Next Steps:**
- Explore the Information Theory demo notebook
- Check out the examples in `metricax/bayesian/examples/`
- Read the comprehensive documentation
- Try MetricaX on your own data!